In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [17]:
import asyncio
import httpx
from lib.tasks import ChatCompletionParams, get_task_results
from lib.temporal_clue import get_temporal_clue_puzzles, get_temporal_clue_tasks
import openai
import os


temporal_clue_tasks = list(get_temporal_clue_tasks())
benchmark_tasks = temporal_clue_tasks[:64]
val_tasks = temporal_clue_tasks[:64]
test_tasks = temporal_clue_tasks[64:128]
train_tasks = temporal_clue_tasks[128:]
len(benchmark_tasks), len(val_tasks), len(test_tasks), len(train_tasks)

(64, 64, 64, 2860)

In [31]:
for key, value in list(get_temporal_clue_puzzles())[2]["solution"].items():
    print(f"{key}. {value}")

A. Mrs. Peacock
B. Poison
C. Studio
D. 12:00 AM
E. Betrayal
F. Drawing Room
G. Studio


In [29]:
print('On a dark winter night, wealthy and enigmatic Mr. John Q. Boddy hosted a small, but lavish, dinner party for some of his closest associates. However, the night ended in tragedy when Mr. Boddy was found dead in one of the rooms of Tudor Mansion in the early hours of the morning. The following persons of interest have been identified as suspects:\n\n• Mrs. White\n• Colonel Mustard\n• Miss Scarlet\n• Madame Rose\n• Mrs. Peacock\n\nAnd the following weapons were found on the premises:\n\n• Poison\n• Horseshoe\n• Revolver\n• Knife\n\nThe murder could only have occured in one of the following rooms:\n\n1. Studio\n2. Carriage House\n3. Conservatory\n4. Drawing Room\n5. Cloak Room\n\nThe rooms are laid out as follows:\n\n  N N  \nW 1|2 E\nW 3|4 E\nW 5|- E\n  S S  \n\nThe exact time of the murder is a bit uncertain, but it has been narrowed down to one of the following times:\n\n• 11:00 PM\n• 11:30 PM\n• 12:00 AM\n\nAt every time the suspects and Mr. Boddy either stayed in their current room or moved to an orthogonally adjacent room (north, south, east, or west). Weapons could be moved by suspects between rooms as well.\n\nEach suspect had one of the following possible motives for killing Mr. Boddy:\n\n• Betrayal (2 suspects)\n• Power (3 suspects)\n\nFor the murder to occur, the murderer and Mr. Boddy must have been alone in a room with at least one weapon at some point in the night. Any clue about Mr. Boddy\'s whereabouts should be read as "Mr. Boddy (dead or alive) ..."\n\nThe available clues are as follows:\n\n- Mrs. Peacock is motivated by Betrayal\n- The Revolver was in the room just west of Colonel Mustard at 12:00 AM\n- Mrs. White moved from the Carriage House to the Drawing Room at 11:30 PM\n- The Knife was in the same room as Madame Rose at 12:00 AM\n- Madame Rose moved from the Carriage House to the Drawing Room at 12:00 AM\n- A suspect motivated by Power was in the same room as the Horseshoe at 11:30 PM\n- The murder weapon was in the Studio at 11:00 PM or Mrs. Peacock was in the Carriage House at 11:00 PM\n- The Revolver was in the room just south of Mr. Boddy at 11:30 PM\n- Madame Rose was in the Carriage House at 11:00 PM\n- Mrs. White was in the Drawing Room at 12:00 AM\n- The Poison was in the room just north of the Revolver at 11:00 PM\n- The Knife was in the Conservatory at 11:00 PM or Colonel Mustard was in the Drawing Room at 11:30 PM\n- A suspect motivated by Betrayal moved from the Conservatory to the Drawing Room at 12:00 AM\n- The Horseshoe and Mrs. Peacock were in the Drawing Room together at least once\n- The Revolver was in the room just south of Mrs. Peacock at 12:00 AM\n- The murderer moved from the Drawing Room to the Conservatory at 11:30 PM\n- Mr. Boddy was in the Studio at 11:00 PM\n- A suspect motivated by Betrayal moved from the Cloak Room to the Conservatory at 11:30 PM\n- The Horseshoe was in the Studio at 12:00 AM or Colonel Mustard was in the Drawing Room at 11:00 PM\n- The Knife was in the same room as Madame Rose at 11:30 PM\n\nPlease answer the following question(s):\n\nA. Who murdered Mr. Boddy?\nB. What weapon did the murderer use?\nC. Where was the murder committed?\nD. When did the murder occur?\nE. Why did the murderer do it?\n\nAnd the following bonus question(s):\n\nF. Where was Madame Rose at 12:00 AM?\nG. Where was the Poison at 11:30 PM?\n\nFill out your final answers in the following format:\n\nA. SUSPECT\nB. WEAPON\nC. ROOM\nD. TIME\nE. MOTIVE\nF. ROOM\nG. ROOM\n\nBest of luck, detective.')

On a dark winter night, wealthy and enigmatic Mr. John Q. Boddy hosted a small, but lavish, dinner party for some of his closest associates. However, the night ended in tragedy when Mr. Boddy was found dead in one of the rooms of Tudor Mansion in the early hours of the morning. The following persons of interest have been identified as suspects:

• Mrs. White
• Colonel Mustard
• Miss Scarlet
• Madame Rose
• Mrs. Peacock

And the following weapons were found on the premises:

• Poison
• Horseshoe
• Revolver
• Knife

The murder could only have occured in one of the following rooms:

1. Studio
2. Carriage House
3. Conservatory
4. Drawing Room
5. Cloak Room

The rooms are laid out as follows:

  N N  
W 1|2 E
W 3|4 E
W 5|- E
  S S  

The exact time of the murder is a bit uncertain, but it has been narrowed down to one of the following times:

• 11:00 PM
• 11:30 PM
• 12:00 AM

At every time the suspects and Mr. Boddy either stayed in their current room or moved to an orthogonally adjacent ro

In [36]:
fireworks = openai.AsyncOpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY"),
)
gemini = openai.AsyncOpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
gemini_semaphore = asyncio.Semaphore(1)
openai_client = openai.AsyncOpenAI(
    http_client=openai.DefaultAsyncHttpxClient(
        timeout=httpx.Timeout(timeout=2400, connect=5.0)
    )
)
openrouter = openai.AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1", api_key=os.getenv("OPENROUTER_API_KEY")
)
together = openai.AsyncOpenAI(
    base_url="https://api.together.xyz/v1", api_key=os.getenv("TOGETHER_API_KEY")
)


results = await asyncio.gather(
    get_task_results(
        tasks=benchmark_tasks,
        client=fireworks,
        model="accounts/fireworks/models/deepseek-r1",
        params=ChatCompletionParams(
            max_tokens=2**17,
            logprobs=True,
            top_logprobs=5,
        ),
        pbar_desc="deepseek-r1",
        # prices=(8.0, 8.0),
        prices=(0.55, 2.19),
    ),
    # get_task_results(
    #     tasks=benchmark_tasks,
    #     client=openrouter,
    #     model="deepseek/deepseek-r1-distill-qwen-1.5b",
    #     pbar_desc="r1-qwen-1.5b",
    #     prices=(0.18, 0.18),
    # ),
    # get_task_results(
    #     tasks=benchmark_tasks,
    #     client=openrouter,
    #     model="qwen/qwen-2.5-7b-instruct",
    #     pbar_desc="qwen-2.5-7b",
    #     params=ChatCompletionParams(
    #         extra_body={"provider": {"order": ["DeepInfra"], "allow_fallbacks": False}},
    #     ),
    #     prices=(0.0025, 0.005),
    # ),
    # get_task_results(
    #     tasks=benchmark_tasks,
    #     client=openrouter,
    #     model="deepseek/deepseek-r1-distill-qwen-14b",
    #     pbar_desc="r1-qwen-14b",
    #     prices=(1.6, 1.6),
    # ),
    # get_task_results(
    #     tasks=benchmark_tasks,
    #     client=openrouter,
    #     model="deepseek/deepseek-r1-distill-qwen-32b",
    #     params=ChatCompletionParams(
    #         extra_body={"provider": {"order": ["DeepInfra"], "allow_fallbacks": False}},
    #     ),
    #     pbar_desc="r1-qwen-32b",
    #     prices=(0.12, 0.18),
    # ),
    # get_task_results(
    #     tasks=benchmark_tasks,
    #     client=openrouter,
    #     model="deepseek/deepseek-r1-distill-llama-70b:free",
    #     params=ChatCompletionParams(
    #         extra_body={"provider": {"order": ["Targon"], "allow_fallbacks": False}},
    #     ),
    #     pbar_desc="r1-llama-70b:targon",
    #     prices=(0.0, 0.0),
    # ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="deepseek/deepseek-r1-distill-llama-70b",
        params=ChatCompletionParams(
            extra_body={"provider": {"order": ["SambaNova"], "allow_fallbacks": False}},
        ),
        pbar_desc="r1-llama-70b:samba",
        prices=(0.7, 1.4),
    ),
    # get_task_results(
    #     tasks=benchmark_tasks,
    #     client=openrouter,
    #     model="deepseek/deepseek-r1-distill-llama-70b",
    #     params=ChatCompletionParams(
    #         extra_body={"provider": {"order": ["Nebius"], "allow_fallbacks": False}},
    #     ),
    #     pbar_desc="r1-llama-70b:nebius",
    #     prices=(0.25, 0.75),
    # ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="openai/o3-mini",
        params=ChatCompletionParams(
            logprobs=False,
        ),
        pbar_desc="o3-mini:medium",
        prices=(1.1, 4.4),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="openai/o3-mini-high",
        params=ChatCompletionParams(logprobs=False),
        pbar_desc="o3-mini:high",
        prices=(1.1, 4.4),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="openai/o1",
        params=ChatCompletionParams(
            logprobs=False,
        ),
        pbar_desc="o1",
        prices=(15, 60),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="anthropic/claude-3.7-sonnet",
        params=ChatCompletionParams(
            max_tokens=128_000,
        ),
        pbar_desc="3.7-sonnet",
        prices=(3, 15),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="anthropic/claude-3.7-sonnet",
        params=ChatCompletionParams(
            max_tokens=128_000,
            extra_body={
                "reasoning": {
                    "max_tokens": 16_000,
                }
            },
        ),
        pbar_desc="3.7-sonnet:16k",
        prices=(3, 15),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="anthropic/claude-3.7-sonnet",
        params=ChatCompletionParams(
            max_tokens=128_000,
            extra_body={
                "reasoning": {
                    "max_tokens": 64_000,
                }
            },
        ),
        pbar_desc="3.7-sonnet:64k",
        prices=(3, 15),
    ),
    # get_task_results(
    #     tasks=benchmark_tasks,
    #     client=gemini,
    #     params=ChatCompletionParams(
    #         logprobs=None,
    #     ),
    #     model="gemini-2.0-pro-exp-02-05",
    #     pbar_desc="gemini-2.0-pro-exp-02-05",
    #     semaphore=gemini_semaphore,
    # ),
    # get_task_results(
    #     tasks=benchmark_tasks,
    #     client=openai_client,
    #     model="o3-mini",
    #     params=ChatCompletionParams(
    #         logprobs=False,
    #         reasoning_effort="high",
    #     ),
    #     pbar_desc="o3-mini:high",
    #     prices=(1.1, 4.4),
    # ),
    # get_task_results(
    #     tasks=benchmark_tasks,
    #     client=openai_client,
    #     model="o3-mini",
    #     params=ChatCompletionParams(
    #         logprobs=False,
    #         reasoning_effort="medium",
    #     ),
    #     pbar_desc="o3-mini:medium",
    #     prices=(1.1, 4.4),
    # ),
)

deepseek-r1:   0%|          | 0/64 [00:00<?, ?it/s]

r1-llama-70b:samba:   0%|          | 0/64 [00:00<?, ?it/s]

o3-mini:medium:   0%|          | 0/64 [00:00<?, ?it/s]

o3-mini:high:   0%|          | 0/64 [00:00<?, ?it/s]

o1:   0%|          | 0/64 [00:00<?, ?it/s]

3.7-sonnet:   0%|          | 0/64 [00:00<?, ?it/s]

3.7-sonnet:16k:   0%|          | 0/64 [00:00<?, ?it/s]

3.7-sonnet:64k:   0%|          | 0/64 [00:00<?, ?it/s]

o3-mini:high: 100%|██████████| 64/64 [00:42<00:00,  4.02it/s, completion_tokens=38369, prompt_tokens=1107, reward=0.567, acc=0.567, spend=$0.00]


In [39]:
for result in results:
    stats = result.stats

    print(
        (
            stats.prompt_tokens * stats.prices[0]
            + stats.completion_tokens * stats.prices[1]
        )
        / stats.grades
        / 1_000_000
    )

0.029131285
0.014573103125
0.0679289359375
0.1700411796875
0.900762421875
0.0166438125
0.22175075806451613
0.392469140625


In [16]:
[exception for result in results[-2] for exception in result.exceptions]

[1 validation error for ChatCompletion
 choices.0.finish_reason
   Input should be 'stop', 'length', 'tool_calls', 'content_filter' or 'function_call' [type=literal_error, input_value='error', input_type=str]
     For further information visit https://errors.pydantic.dev/2.10/v/literal_error,
 1 validation error for ChatCompletion
 choices.0.finish_reason
   Input should be 'stop', 'length', 'tool_calls', 'content_filter' or 'function_call' [type=literal_error, input_value='error', input_type=str]
     For further information visit https://errors.pydantic.dev/2.10/v/literal_error]

In [34]:
results[-6][-9].exceptions[0]

1 validation error for ChatCompletion
choices.0.finish_reason
  Input should be 'stop', 'length', 'tool_calls', 'content_filter' or 'function_call' [type=literal_error, input_value='error', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error

In [37]:
results[-1][-9].exceptions[0]#.body

TypeError("AsyncCompletions.create() got an unexpected keyword argument 'reasoning'")

In [5]:
results[-1][-15].exceptions

[openai.RateLimitError("Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}")]

In [23]:
await get_task_results(
    tasks=benchmark_tasks,
    client=openrouter,
    model="deepseek/deepseek-r1-distill-llama-70b",
    # params=ChatCompletionParams(
    #     extra_body={
    #         "provider": {"order": ["Together"], "allow_fallbacks": False}
    #     },
    # ),
    pbar_desc="r1-llama-70b",
    prices=(2.0, 2.0),
)

r1-llama-70b:   0%|          | 0/100 [00:00<?, ?it/s]

([TaskResult(task=Task(messages=[{'role': 'user', 'content': 'On a dark winter night, wealthy and enigmatic Mr. John Q. Boddy hosted a small, but lavish, dinner party for some of his closest associates. However, the night ended in tragedy when Mr. Boddy was found dead in one of the rooms of Tudor Mansion in the early hours of the morning. The following persons of interest have been identified as suspects:\n\n• Miss Peach\n• Monsieur Brunette\n• Mr. Green\n• Professor Plum\n• Mrs. White\n• Colonel Mustard\n• Miss Scarlet\n• Mrs. Peacock\n• Sgt. Gray\n• Madame Rose\n\nAnd the following weapons were found on the premises:\n\n• Candlestick\n• Wrench\n• Lead Pipe\n• Revolver\n• Poison\n• Knife\n• Rope\n• Horseshoe\n\nThe murder could only have occured in one of the following rooms:\n\n01. Studio\n02. Gazebo\n03. Lounge\n04. Drawing Room\n05. Library\n06. Trophy Room\n07. Cloak Room\n08. Courtyard\n09. Kitchen\n10. Fountain\n11. Dining Room\n12. Carriage House\n13. Ballroom\n\nThe rooms are 